In [1]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MultiLabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
import matplotlib.pyplot as plt

import json
import glob

import pickle

In [3]:
pd.set_option('display.float_format', '{:.4f}'.format)

In [4]:
all_data = []
for file in glob.glob("../data/processed/*.json"):
    print(file)
    with open(file, "r") as f:
        all_data.extend(json.load(f))

df = pd.DataFrame(all_data)
df = df.dropna(thresh=8)
df = df.reset_index(drop=True)

../data/processed\channels0_99.json
../data/processed\channels100_.json
../data/processed\channels3425_.json
../data/processed\channels6727_.json
../data/processed\channelsa10034_.json
../data/processed\channelsa13313_.json
../data/processed\channels_missing.json


In [5]:
df["created_date"] = pd.to_datetime(df["created_date"], format="ISO8601")

In [6]:
time_cols = ["created_date"]
num_cols = ["avg_duration_seconds", "avg_seconds_between_uploads"]
basic_cat_cols = ["country", "most_common_video_genre"]
multi_cat_cols = ["category", "all_video_genres"]
text_cols = ["description", "aggregated_tags", "recent_video_titles"]
drop_cols = ["channel_id", "channel_name", "defaultLanguage"]
set(time_cols + num_cols + basic_cat_cols + multi_cat_cols + text_cols + drop_cols) == set(df.columns)

True

TODO: investigate below scaling step

In [9]:
df["avg_seconds_between_uploads"] = df["avg_seconds_between_uploads"].fillna(df["avg_seconds_between_uploads"].max())

In [10]:
class DateTimeToPosix(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return (pd.to_datetime(X.iloc[:, 0]).astype(int) // 10**9).values.reshape(-1, 1)

In [11]:
# MLB does not have fit/transform. must wrap in custom transformer, give it fit/transform, and col transformer will accept
class MultiLabelBinarizerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()
    
    def fit(self, X, y=None):
        self.mlb.fit(X.iloc[:, 0])
        return self
    
    def transform(self, X, y=None):
        return self.mlb.transform(X.iloc[:, 0])


In [ ]:
ct = make_column_transformer(
    (DateTimeToPosix(), time_cols),
    (StandardScaler(), num_cols),
    (make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder()), basic_cat_cols), #TODO: almost certainly a better way to impute countries
    (MultiLabelBinarizerTransformer(), ["category"]),
    (MultiLabelBinarizerTransformer(), ["all_video_genres"]),
    (TODO, text_cols)
    ("drop", drop_cols)
)